In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from scipy.optimize import linear_sum_assignment

In [2]:
train_cat = pd.read_csv("datasets/mhmm/final_train.csv")
test_cat = pd.read_csv("datasets/mhmm/final_test.csv")

train_gauss = pd.read_csv("datasets/ghmm/final_train.csv")
test_gauss = pd.read_csv("datasets/ghmm/final_test.csv")

In [16]:
USE_TEMPORAL_FEATURES = False
observation_col = "observation_code_w_temporal" if USE_TEMPORAL_FEATURES else "observation_code"

In [17]:
X_train_cat = train_cat[observation_col].values.reshape(-1, 1)
X_test_cat = test_cat[observation_col].values.reshape(-1, 1)

In [18]:
model_dir = hmm.MultinomialHMM(
    n_components=2,
    n_iter=100,
    algorithm="viterbi",
    random_state=42,
    params="ste"
)

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [19]:
# Initialize and train
model_dir.emissionprob_ = np.ones((2, train_cat[observation_col].nunique())) / train_cat[observation_col].nunique()
model_dir.fit(X_train_cat, lengths=[len(X_train_cat)])

MultinomialHMM(n_components=2, n_iter=100,
               n_trials=array([ 12,   6,   3, ..., 214, 214, 211]),
               random_state=RandomState(MT19937) at 0x1D5748C8240)

In [20]:
n_states_trend = 6
model_trend = hmm.MultinomialHMM(
    n_components=n_states_trend,
    n_iter=100,
    algorithm="viterbi",
    random_state=42,
    params="ste"
)

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [21]:
model_trend.emissionprob_ = np.ones((n_states_trend, train_cat[observation_col].nunique())) / train_cat[observation_col].nunique()
model_trend.fit(X_train_cat, lengths=[len(X_train_cat)])

MultinomialHMM(n_components=6, n_iter=100,
               n_trials=array([ 12,   6,   3, ..., 214, 214, 211]),
               random_state=RandomState(MT19937) at 0x1D5748C8340)

# Model Evaluation

In [22]:
def evaluate(y_true, y_pred, target_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    
    print(f"{target_name} Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print()

In [23]:
# Predict and align states
pred_states_dir = model_dir.predict(X_test_cat)
cm_dir = confusion_matrix(test_cat["Direction_quarter"], pred_states_dir)
row_ind_dir, col_ind_dir = linear_sum_assignment(-cm_dir)
mapped_pred_dir = np.zeros_like(pred_states_dir)
for i, j in zip(row_ind_dir, col_ind_dir):
    mapped_pred_dir[pred_states_dir == j] = i

In [24]:
pred_states_trend = model_trend.predict(X_test_cat)
cm_trend = confusion_matrix(test_cat["Trend_class"], pred_states_trend)
row_ind, col_ind = linear_sum_assignment(-cm_trend)
mapped_pred_trend = np.zeros_like(pred_states_trend)
for i, j in zip(row_ind, col_ind):
    mapped_pred_trend[pred_states_trend == j] = i

# Performance Not Considering Temporal Features

In [25]:
evaluate(test_cat['Direction_quarter'], mapped_pred_dir, 'Direction_quarter')
evaluate(test_cat['Trend_class'], mapped_pred_trend, 'Trend_class')

Direction_quarter Evaluation:
Accuracy: 0.68
Precision: 0.78
Recall: 0.68
F1 Score: 0.55

Trend_class Evaluation:
Accuracy: 0.46
Precision: 0.46
Recall: 0.46
F1 Score: 0.45



In [26]:
print(classification_report(test_cat['Direction_quarter'], mapped_pred_dir, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00       702
           1       0.68      1.00      0.81      1468

    accuracy                           0.68      2170
   macro avg       0.84      0.50      0.41      2170
weighted avg       0.78      0.68      0.55      2170



In [27]:
print(classification_report(test_cat['Trend_class'], mapped_pred_trend, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00        39
           3       0.00      0.00      0.00       119
           4       0.31      0.51      0.38       659
           5       0.61      0.50      0.55      1317

    accuracy                           0.46      2170
   macro avg       0.15      0.17      0.16      2170
weighted avg       0.46      0.46      0.45      2170



# Performance Considering Temporal Features

In [13]:
evaluate(test_cat['Direction_quarter'], mapped_pred_dir, 'Direction_quarter')
evaluate(test_cat['Trend_class'], mapped_pred_trend, 'Trend_class')

Direction_quarter Evaluation:
Accuracy: 0.68
Precision: 0.78
Recall: 0.68
F1 Score: 0.55

Trend_class Evaluation:
Accuracy: 0.46
Precision: 0.46
Recall: 0.46
F1 Score: 0.45



In [14]:
print(classification_report(test_cat['Direction_quarter'], mapped_pred_dir, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00       702
           1       0.68      1.00      0.81      1468

    accuracy                           0.68      2170
   macro avg       0.84      0.50      0.41      2170
weighted avg       0.78      0.68      0.55      2170



In [15]:
print(classification_report(test_cat['Trend_class'], mapped_pred_trend, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00        39
           3       0.00      0.00      0.00       119
           4       0.31      0.51      0.38       659
           5       0.61      0.50      0.55      1317

    accuracy                           0.46      2170
   macro avg       0.15      0.17      0.16      2170
weighted avg       0.46      0.46      0.45      2170



# Gaussian HMM

In [35]:
features_gaussian = ['Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
X_train_gauss = train_gauss[features_gaussian].values
X_test_gauss = test_gauss[features_gaussian].values

In [36]:
train_gauss[features_gaussian]

,Price,Open,High,Low,Vol.,Change %
0,-0.475003,-0.475112,-0.475089,-0.474969,0.437682,-22.465808
1,-0.475242,-0.475351,-0.475325,-0.475210,-0.432551,-1.756767
2,-0.475481,-0.475590,-0.475561,-0.475452,-0.640796,-1.900533
3,-0.475242,-0.475351,-0.475325,-0.475210,-0.698059,2.012598
4,-0.475242,-0.475351,-0.475325,-0.475210,-0.736621,-0.029329
...,...,...,...,...,...,...
8674,0.269707,0.267182,0.266866,0.270457,-0.432908,0.020091
8675,0.283795,0.278653,0.276084,0.283255,-0.295048,0.392984
8676,0.275915,0.282955,0.275612,0.280358,-0.455528,-0.260702
8677,0.273289,0.272201,0.271593,0.275287,-0.382798,-0.107951


In [37]:
model_gauss_dir = hmm.GaussianHMM(
    n_components=2,
    covariance_type="diag",
    n_iter=100,
    random_state=42
)
model_gauss_dir.fit(X_train_gauss)

# Predict and align states
pred_states_gauss_dir = model_gauss_dir.predict(X_test_gauss)
cm_gauss_dir = confusion_matrix(test_gauss["Direction_quarter"], pred_states_gauss_dir)
row_ind_gauss_dir, col_ind_gauss_dir = linear_sum_assignment(-cm_gauss_dir)
mapped_pred_gauss_dir = np.zeros_like(pred_states_gauss_dir)
for i, j in zip(row_ind_gauss_dir, col_ind_gauss_dir):
    mapped_pred_gauss_dir[pred_states_gauss_dir == j] = i

# Calculate probabilities and log loss
prob_matrix_dir = model_gauss_dir.predict_proba(X_test_gauss)
prob_matrix_dir = prob_matrix_dir[:, col_ind_gauss_dir]

Model is not converging.  Current: 80322.46966926738 is not greater than 80322.81097285444. Delta is -0.3413035870617023


In [38]:
model_gauss_trend = hmm.GaussianHMM(
    n_components=6,
    covariance_type="diag",
    n_iter=100,
    random_state=42
)
model_gauss_trend.fit(X_train_gauss)

# Predict and align states
pred_states_gauss_trend = model_gauss_trend.predict(X_test_gauss)
cm_gauss_trend = confusion_matrix(test_gauss["Trend_class"], pred_states_gauss_trend)
row_ind_gauss, col_ind_gauss = linear_sum_assignment(-cm_gauss_trend)
mapped_pred_gauss_trend = np.zeros_like(pred_states_gauss_trend)
for i, j in zip(row_ind_gauss, col_ind_gauss):
    mapped_pred_gauss_trend[pred_states_gauss_trend == j] = i

# Calculate probabilities and log loss
prob_matrix_trend = model_gauss_trend.predict_proba(X_test_gauss)
prob_matrix_trend = prob_matrix_trend[:, col_ind_gauss]

Model is not converging.  Current: 113704.1787652482 is not greater than 113704.29512406125. Delta is -0.1163588130439166


# Performance Considering Temporal Features

In [32]:
evaluate(test_gauss['Direction_quarter'], mapped_pred_gauss_dir, 'Direction_quarter')
evaluate(test_gauss['Trend_class'], mapped_pred_gauss_trend, 'Trend_class')

Direction_quarter Evaluation:
Accuracy: 0.68
Precision: 0.46
Recall: 0.68
F1 Score: 0.55

Trend_class Evaluation:
Accuracy: 0.61
Precision: 0.57
Recall: 0.61
F1 Score: 0.46



In [33]:
print(classification_report(test_gauss['Direction_quarter'], mapped_pred_gauss_dir, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       702
           1       0.68      1.00      0.81      1468

    accuracy                           0.68      2170
   macro avg       0.34      0.50      0.40      2170
weighted avg       0.46      0.68      0.55      2170



In [34]:
print(classification_report(test_gauss['Trend_class'], mapped_pred_gauss_trend, zero_division=0))

              precision    recall  f1-score   support

           0       0.33      0.25      0.29         4
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00        39
           3       0.00      0.00      0.00       119
           4       0.67      0.00      0.01       659
           5       0.61      1.00      0.76      1317

    accuracy                           0.61      2170
   macro avg       0.27      0.21      0.17      2170
weighted avg       0.57      0.61      0.46      2170

